# Ternviz

Make a video of a turning structure from SMILES.

Checkout the Twitter bot [@ternviz](https://twitter.com/ternviz)!

In [ ]:
# @title Install Required Package
!pip install -q git+https://github.com/whitead/ternviz.git condacolab
import condacolab

condacolab.install()
print("📦Installing Conda Packages")
!mamba install -q -c conda-forge --file https://raw.githubusercontent.com/whitead/ternviz/main/requirements.txt > /dev/null

In [ ]:
import ternviz

smiles1 = "CCO"
smiles2 = ""
names = []

if smiles2:
    smiles = [smiles1, smiles2]
    names = []

ternviz.main()